Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and student ID below:

In [ ]:
NAME1 = ""
NAME2 = ""
NAME3 = ""
ID1 = "" ## Your student id
ID2 = ""
ID3 = ""

---

# Lab1 Signal Processing Basics

As we may use some basic signal processing techniques, such as fft, filtering, etc. in our lab. We may equip you with the basic signal processing background.

In [ ]:
import numpy as np
from scipy import integrate 
from scipy import signal
from matplotlib import pyplot as plt

DFT(discrete Fourier Transform) plays an extreme role in all kinds of signal processing tasks. It is given by

$$
    X[k] = \sum_{n=0}^{N-1} x[n] \exp{ \left(-j 2 \pi \dfrac{nk}{N}\right)}
$$
where $x[n]$ is the input sequence with $N$ samples.
It is used for filtering, spectral analysis, complexity reduction, equalization and so many other kinds of analysis. Before that, let us first start with CFT (continuous Froutier Transform).

## 1.1 CFT

In [ ]:
def cft(g, f):
    """Numerically evaluate the Fourier Transform of g for the given frequencies"""    
    result = np.zeros(len(f), dtype=complex)
    
    # Loop over all frequencies and calculate integral value
    for i, ff in enumerate(f):
        # Evaluate the Fourier Integral for a single frequency ff, 
        # assuming the function is time-limited to abs(t)<5
        result[i] = complex_quad(lambda t: g(t)*np.exp(-2j*np.pi*ff*t), -10, 10)
    return result

def complex_quad(g, a, b):
    """Return definite integral of complex-valued g from a to b, 
    using Simpson's rule"""
    # 2501: Amount of used samples for the trapezoidal rule
    t = np.linspace(a, b, 2501)  
    x = g(t)
    return integrate.simps(y=x, x=t)

def triang(t):
    return (1-abs(t)) * (abs(t)<1).astype(float)
Fs = 10000
t = np.arange(-10, 10, 1/Fs)

f = np.arange(-3, 3, 1/20.)

fig,ax = plt.subplots(2,1,figsize=(6,6))
ax[0].plot(t, triang(t))
ax[0].set_xlabel('t')
ax[0].set_ylabel('g(t)')
ax[0].set_title('Triangular function')
ax[0].set_xlim(-10,10)
ax[0].set_ylim(0,1)
ax[0].grid()

cft_triang = cft(triang, f)
ax[1].plot(f, np.abs(cft_triang))
ax[1].set_xlabel('f')
ax[1].set_ylabel('|G(f)|')
ax[1].set_title('Fourier Transform of Triangular function')
ax[1].set_xlim(-3,3)
ax[1].set_ylim(0,1)
ax[1].grid()

fig.tight_layout()

The CFT of the triangle is a sinc-function. If we use a periodic triangle, what would happen？

In [ ]:
def periodic_triang(t):
    return triang(np.mod(t+1, 2)-1)

fig,ax = plt.subplots(2,1, figsize=(6,6))
# ax[0].subplot(121)
ax[0].plot(t, periodic_triang(t))
ax[0].set_xlabel("t")
ax[0].grid()
ax[0].set_ylim(0,1)
ax[0].set_xlim(-10,10)
ax[0].set_title("Periodic Triangular function")

# plt.subplot(122)
# Energy correction /10, since the periodic signal has 10 periods (i.e. 10 times the power)
cft_periodic_triang = cft(periodic_triang, f) / 10
ax[1].plot(f, abs(cft_periodic_triang), '-', label='$X(f)$ (periodic)')
ax[1].plot(f, abs(cft_triang), label='$X(f)$ non-periodic', lw=2); 
ax[1].set_xlabel("f")
ax[1].set_ylim(0,1)
ax[1].set_xlim(-3,3)
ax[1].set_title("Fourier Transform of Periodic Triangular function")
ax[1].grid()
ax[1].legend()
fig.tight_layout()

As shown,the spectrum of the periodic signal become not that smooth. It follows the shape of the non-perioc one. To conclude, the spectrum of the periodic signal is a sampled version of the continuous spectrum. This is one crucial observation. Furthermore, if we look into when the "sharp" will appear, you may find it is the countdown of the time-domain period $T$. To be exact, the spectrum is non-zero every $1/T = 0.5$Hz. 

## 1.2 Windowing

Next we will discuss the windowing. What will happen if we add windows to a periodic signal.

In [ ]:
def rect(t):
    return (abs(t)<0.5).astype(float)
window = lambda t: rect((t-1)/4)
def windowed_periodic_triang(t):
    return periodic_triang(t) * window(t)

# plt.subplot(121)
fig, ax = plt.subplots(2,1, figsize=(6,6))
ax[0].plot(t, windowed_periodic_triang(t), 'b-', label='windowed', lw=2);
ax[0].plot(t, window(t), 'r-', label='window')
ax[0].plot(t, periodic_triang(t), 'g-x', label='periodic', markevery=2000)
ax[0].set_xlabel("t")
ax[0].grid()
ax[0].set_ylim(-0.2,1.6)
ax[0].set_xlim(-5,5)
ax[0].set_title("Windowed Periodic Triangular function")
ax[0].grid()
ax[0].legend()

cft_windowed_periodic_triang = cft(windowed_periodic_triang, f) / 2
cft_window = cft(window, f) / 4
# plt.subplot(122)
ax[1].plot(f, abs(cft_windowed_periodic_triang), 'b-', label='windowed', lw=2);
ax[1].plot(f, abs(cft_periodic_triang), 'g-x', label='periodic')
ax[1].plot(f, abs(cft_window), 'r-', label='window')
ax[1].set_xlabel("f")
ax[1].set_ylim(0,1)
ax[1].set_xlim(-1.5,1.5)
ax[1].set_title("Fourier Transform of Windowed Periodic Triangular function")
ax[1].grid()
ax[1].legend()
fig.tight_layout()

See, it becomes not that "sharp" again. It is obvious since the input signal becomes non-periodic. Oberserve carefully the windowed spectrum and the original. It crosses every non-zero intersection points of the non-windowed one. You may develop a sense that the windowing process is exactly doing multiplication in time domain. Mathematically, it will give rise the convoluntion of the frequency domain.

In [ ]:
window = lambda t: rect(t/2)
def windowed_periodic_triang(t):
    return periodic_triang(t) * window(t)

fig, ax = plt.subplots(2,1, figsize=(6,6))
ax[0].plot(t, windowed_periodic_triang(t), 'b-', label='windowed', lw=2);
ax[0].plot(t, window(t), 'r-', label='window')
ax[0].plot(t, periodic_triang(t), 'g-x', label='periodic', markevery=2000)
ax[0].set_xlabel("t")
ax[0].grid()
ax[0].set_ylim(-0.2,1.6)
ax[0].set_xlim(-5,5)
ax[0].set_title("Windowed Periodic Triangular function")
ax[0].grid()
ax[0].legend()

cft_windowed_periodic_triang = cft(windowed_periodic_triang, f) / 2
cft_window = cft(window, f) / 4
# plt.subplot(122)
ax[1].plot(f, abs(cft_windowed_periodic_triang), 'b-', label='windowed', lw=2);
ax[1].plot(f, abs(cft_periodic_triang), 'g-x', label='periodic')
ax[1].plot(f, abs(cft_window), 'r-', label='window')
ax[1].set_xlabel("f")
ax[1].set_ylim(0,1)
ax[1].set_xlim(-1.5,1.5)
ax[1].set_title("Fourier Transform of Windowed Periodic Triangular function")
ax[1].grid()
ax[1].legend()
fig.tight_layout()

Naturally, when windowing the periodic function to just one period, it becomes the original (non-periodic) function. Hence, the spectrum of the windowed periodic (blue curve) and non-periodic function (black crosses) are equal. The blue spectrum is the convolution of the red curve (window) with the green curve (periodic function).



## 1.3 DFT and FFT

The Discrete Fourier Transform (DFT) assumes that its input signal is one period of a periodic signal. Its outputs are the discrete frequencies of this periodic signal. For example, if we input an non-periodic signal, the DFT would transform the input as the periodic one.

In common practice, we use FFT to compute DFT. Fast Fourier Transform (FFT) is just an algorithm for fast and efficient computation of the DFT. In our context, we then no longer distinguish between DFT and FFT.

In [ ]:
n = np.arange(32)
xn = np.linspace(0, 1, len(n))

fig, ax = plt.subplots(2,1, figsize=(6,6))
ax[0].stem(n, xn);
ax[0].set_xlabel("n")
ax[0].grid()
ax[0].set_ylim(-0.0,1.1)
ax[0].set_xlim(-1,33)
ax[0].set_title("Input sequence")
ax[0].grid()
# ax[0].legend()

xn_periodic = np.hstack([xn]*5)
# plt.subplot(122)
ax[1].stem(xn_periodic)
ax[1].set_xlabel("f")
ax[1].set_ylim(0,1.1)
ax[1].set_xlim(0,161)
ax[1].set_title("Siganl assumed by DFT")
ax[1].grid()
# ax[1].legend()
fig.tight_layout()

Therefore, the outcome of DFT would be discrete.  Suppose we input sequence $x[n]$ with $N$ samples, sampling rate $f_s$. Then what is the frequency intervals $\Delta_f$? In other word, what frequencies will be output in DFT frequency domain? 

Mathematically, the frequency bins interval are given by
$$
\Delta_f = \frac{1}{T} = \frac{f_s}{N}
$$

In other word, FFT output frequency bins correspond to the frequencies
$$
f_k = k \cdot \frac{fs}{N}
$$

In [ ]:
n = np.arange(32)
xn = np.linspace(0, 1, len(n))

yn = np.fft.fft(xn)
# y_freq = np.fft.fftfreq(y_fft.shape[0],d = len(n))
y_abs = np.abs(yn)

xn_periodic = np.hstack([xn]*5)
yn_p = np.fft.fft(xn_periodic)
yn_p_abs = np.abs(yn_p)
fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0][0].stem(n, xn);
ax[0][0].set_xlabel("n")
ax[0][0].grid()
ax[0][0].set_ylim(0,1.0)
ax[0][0].set_xlim(0,30)
ax[0][0].set_title("Input sequence")
ax[0][0].grid()

ax[0][1].stem(xn_periodic)
ax[0][1].set_xlabel("f")
ax[0][1].set_ylim(0,1.1)
ax[0][1].set_xlim(0,161)
ax[0][1].set_title("Siganl assumed by DFT")
ax[0][1].grid()


ax[1][0].stem(y_abs);
ax[1][0].set_xlabel("n")
ax[1][0].grid()
ax[1][0].set_ylim(0,20)
ax[1][0].set_xlim(-1,33)
ax[1][0].set_title("DFT of Input sequence")
ax[1][0].grid()

ax[1][1].stem(yn_p_abs);
ax[1][1].set_xlabel("n")
ax[1][1].grid()
ax[1][1].set_ylim(0,85)
ax[1][1].set_xlim(-4,165)
ax[1][1].set_title("DFT of Periodic Sequence")
ax[1][1].grid()

fig.tight_layout()

See, the spectrum looks familar. You can find out the $\Delta_f$ and the pattern of non-zero frequencies.

Mathematically, FFT will derive the frequency of the input signals. We clearly see it by generating series of cosine signals. 

In [ ]:
f = 50
fs = 440
x = np.linspace(0,1,fs)
y = 2 * np.cos(2 * np.pi * f * x)

fig, ax = plt.subplots(2,1, figsize=(6,6))
ax[0].plot(x, y);
ax[0].set_xlabel("x")
ax[0].set_ylabel("y")
ax[0].grid()
ax[0].set_ylim(-6,6)
ax[0].set_xlim(0,1)
ax[0].set_title("Time-domain signals")
ax[0].grid()

y_fft = np.fft.fft(y)
y_abs = np.abs(y_fft)
ax[1].plot(y_abs)
ax[1].set_xlabel("f")
# ax[1].set_ylim(0,1)
# ax[1].set_xlim(0,160)
ax[1].set_title("Frequency-domain spectrum")
ax[1].grid()
# ax[1].legend()
fig.tight_layout()

After performing FFt on cosine signal, we can see claerly two peaks in the spectrum: $f_{p1}=50$Hz and $f_{p2}= f - f_{p1} = 390$ Hz. The first peak shows the frequency of the input signals, which is exactly 50Hz. The other peak, however, is symmetrical to the first peak. This is because the FFT contains only half of the information. 

Moreover, you need to take a look at the sampling frequency `fs`. What will happen if we increase or decrease it?

You can change the frequency and observe it on your own. 


Up to now, you have already known how we can derive frequencies of input signals. Next you will be required to finish a function which will do that. Just try it!

<b><font color="red" size=5>Checkpoints (20 points)</font></b>

You have to implement a function `show_freq()`, which will compute the frequencies in the given signals using FFT. You should return an 1-d numpy array containing all the frequencies the `input_signal` has. You are blind to our test code, which means you shall use what we taught just now to implement.

For example, if the input signal is $y = \sin(2 \pi \times 200 x) + \cos(2 \pi \times 100 x)$, then you should return `array([100,200])`. Orders do not matter. You should guarantee that you output the right frequencies the signals contain. You can write a few test on your own. After you submit your code, we will use our test code to autograde your implementation.

In [ ]:
def show_freq(input_signals,fs):
    '''
        - Description: You have to compute the frequencies the input signal contains
        - Input:
            - input_signals: 1-d ndarray
            - fs: sampling frequency
        - Output:
            - freq: ndarray of frequencies
    
    '''
    
    # YOUR CODE HERE
    raise NotImplementedError()
    

## 1.4 Zero padding and Frequency resolution

Zero padding is a simple concept; it simply refers to adding zeros to end of a time-domain signal to increase its length. There are a few reasons why you might want to zero pad time-domain data. The most common reason is to make a waveform have a power-of-two number of samples. When the time-domain length of a waveform is a power of two, radix-2 FFT algorithms, which are extremely efficient, can be used to speed up processing time. FFT algorithms made for FPGAs also typically only work on lengths of power two.

While it’s often necessary to stick to powers of two in your time-domain waveform length, it’s important to keep in mind how doing that affects the resolution of your frequency-domain output. Back to the previous problem, suppose we input sequence $x[n]$ with $N$ samples, sampling rate $f_s$. At the end of the signal, we pad zero with length $N_{p}$. What is the resolution of the spectrum?

To be exact, there are two aspects that we need to take into consideration. The first one is “real resolution”, and another one is FFT resolution. Without zero padding, this two resolution is the same. This is confusing when, however, we perform zero-padding. The conclusion is zero-padding will not increase the real resolution of the output, since no useful information is increased. However, zero-padding would exactly increase the FFT resoluton. We will explain it in the following part.

Since we have $N$ signals with sampling rate of $f_s$, the real time of the signal would be 
$$
T = \frac{N}{f_s}
$$

Hereby, the real resolution of the frequency is 
$$
\Delta_f^{\text{real}} = \frac{1}{T} = \dfrac{f_s}{N}
$$

The real resolution refers to the waveform frequency that you can derive. With a small real resolution, you are not able to show frequencies with tiny differences, i.e 120Hz and 121Hz. In other word, you can not separate two tones at nearby frequencies , no matter how much zero is padded.

After padding, the FFT length of signals will be increased, i.e.
$$
T_{p} = T + \frac{N_p}{f_s}
$$

Then, the FFT resolution is given by
$$
\Delta_f^{\text{FFT}} = \frac{1}{T_p} = \dfrac{f_s}{N + N_p}
$$

That is to say, by performing zero-padding, more points can be plotted. You can understand FFt resolution as DPI (plot points per inch) resolution. This is in line with the statement, that ZP does not reveal extra information from the spectrum. Instead it merely interpolates the coarse spectrum to become more smooth. It does not help in distinguishing between two close frequencies.


In [ ]:
f1 = 50
f2 = 50.8
x=np.linspace(0,1,600)      
y = 2 * np.cos(2 * np.pi * f1 *x) + 3 * np.cos(2 * np.pi * f2 * x)

y_fft = np.fft.fft(y)
y_fft_len = (int)(y_fft.shape[0] / 2)
y_fft_freq = np.fft.fftfreq(y_fft.shape[0],d = 1/600)
y_abs = np.abs(y_fft)

y_fft_p = np.fft.fft(y,512)
y_fft_len_p = (int)(y_fft_p.shape[0] / 2)
y_fft_freq_p = np.fft.fftfreq(y_fft_p.shape[0],d = 1/600)
y_abs_p = np.abs(y_fft_p)

x_p = np.linspace(0,3,600)
y_p = 2 * np.cos(2 * np.pi * f1 *x_p) + 3 * np.cos(2 * np.pi * f2 * x_p)

y_fft_yp = np.fft.fft(y_p)
y_fft_len_yp = (int)(y_fft_yp.shape[0] / 2)
y_fft_freq_yp = np.fft.fftfreq(y_fft_yp.shape[0], d = 1/200)
y_abs_yp = np.abs(y_fft_yp)

fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0][0].plot(x,y);
ax[0][0].set_xlabel("t")
ax[0][0].set_ylabel("y")
# ax[0].set_ylim(-6,6)
# ax[0].set_xlim(0,1)
ax[0][0].set_title("Raw Signal")
ax[0][0].grid()

ax[0][1].plot(y_fft_freq[:y_fft_len],y_abs[:y_fft_len]);
ax[0][1].set_xlabel("f")
ax[0][1].set_ylabel("|G(f)|")
# ax[0].set_ylim(-6,6)
ax[0][1].set_xlim(0,100)
ax[0][1].set_title("FFT without zero padding")
ax[0][1].grid()

ax[1][0].plot(y_fft_freq_yp[:y_fft_len_yp],y_abs_yp[:y_fft_len_yp]);
ax[1][0].set_xlabel("f")
ax[1][0].set_ylabel("|G(f)|")
# ax[0].set_ylim(-6,6)
ax[1][0].set_xlim(20,80)
ax[1][0].set_title("FFT with increase samples")
ax[1][0].grid()

ax[1][1].plot(y_fft_freq_p[:y_fft_len_p],y_abs_p[:y_fft_len_p]);
ax[1][1].set_xlabel("f")
ax[1][1].set_ylabel("|G(f)|")
# ax[0].set_ylim(-6,6)
ax[1][1].set_xlim(0,100)
ax[1][1].set_title("FFT with zero padding")
ax[1][1].grid()


Up to now, I believe you have understood a little bit about the signal processing. It is a good start. In the next few sections, we would guide you through a hand-on distance measuring with acoustic sensing. Go ahead!